In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [39]:
import glob 
import os

import numpy as np
import json
from PIL import Image
import matplotlib.pyplot as plt 

In [40]:
import numba
from numba import types
from numba.typed import Dict,List
import math

In [41]:
import time
from numba import jit,njit
from numba import prange

In [42]:
from numba.types import List
from functools import reduce



In [3]:
img_dir = "./Lab1_data/alphabet/"
input_dir = "./Lab1_data/input/"
test_fns = glob.glob(img_dir + "*.png")
input_fns = glob.glob(input_dir  + "*.png")
with open("./Lab1_data/frequencies.json") as f :
    freq = json.load(f)
    
alphabet = [os.path.basename(test_fn).split('.')[0] for test_fn in test_fns]
icon = [np.array(Image.open(test_fn),int) for test_fn in test_fns]
etalon = {alphabet[i]: icon[i] for i in range(len(alphabet))}
input_images = [np.array(Image.open(input_fns),int) for input_fns in input_fns]
input_names = [os.path.basename(input_fns).split('.')[0] for input_fns in input_fns]
input_dict = {input_names[i]:input_images[i] for i in range(len(input_names))}


S1 = 0
S2 = 0
prob = {i:freq[i] for i in list(freq.keys()) if " "  in i} 
bigraph = {i:freq[i] for i in list(freq.keys()) if " " not in i}


for i in prob.keys():
    if i[0] == " ":
        S1 += freq[i]
    else :
        S2 += freq[i]
            
for i in prob.keys():
    if i[0] == " ":
        prob[i] = prob[i] / S1
    if i[1] == " ":
        prob[i] = prob[i] / S2
    
for f in bigraph.keys():
    ss = 0
    for g in bigraph.keys():
        if f[0] == f[0]:
            ss += bigraph[g]
    bigraph[f] = bigraph[f]/ss

In [4]:

@jit(nopython = True)
def logical_xor(a, b,c  = None):
    if c == None:
        return bool((a ^ b) and not(a and b))
    else :
        return (a ^ b ^ c) and not(a and b and c)

@jit(nopython=True)
def prod(n):
    shp = [n.shape[0], n.shape[1]]
   
    result1 = 1.0
    for i in range(shp[0]):
        for j in range(shp[1]):
            result1  =  result1 *  n[i][j]

    return result1

@jit(nopython = True)
def logical_xor(a, b,c  = None):
    if c == None:
        return bool((a ^ b) and not(a and b))
    else :
        return (a ^ b ^ c) and not(a and b and c)

@jit(nopython=True)
def prod(n):
    shp = [n.shape[0], n.shape[1]]
   
    result1 = 1.0
    for i in range(shp[0]):
        for j in range(shp[1]):
            result1  =  result1 *  n[i][j]

    return result1

@jit(nopython = True)
def prob_XOR(imgx,imgk,probk):
    P = np.zeros((imgx.shape[0],imgx.shape[1]))
    for i in range(imgx.shape[0]):
        for j in range(imgx.shape[1]):
            P[i][j] = (probk**(logical_xor(imgx[i][j],imgk[i][j])) * (1 - probk)**(logical_xor(1,imgx[i][j],imgk[i][j])))
    
    return P

In [5]:

@jit(fastmath=True,nopython=True)
def vers3_dict1(input_image,index,etalon):
    index = index
    input_img_shape = input_image.shape[1]
    keys = list(etalon.keys())
    for j in (range((len(keys)))):
        img_k = etalon[keys[j]].shape[1]
        input_t = input_img_shape - img_k
        if input_t - 12 > 0:
            if index[input_img_shape - 1][j]  == 0:
              img_t = input_image[:,:img_k + 1]
              P  = prob_XOR(img_t,etalon[keys[j]],0.3)
              val = np.log(prod(P))
              index[input_img_shape - 1][j] = val
              input_image = input_image[:,img_k+1:]
              index = vers3_dict1(input_image,index,etalon)
          else:
            continue 
        elif input_t == 0:
            if index[input_img_shape - 1][j] == 0:
               img_t = input_image[:,:img_k + 1]
               P  = prob_XOR(img_t,etalon[keys[j]],0.3)
               val = np.log(prod(P))
               index[input_img_shape - 1][j] = val
            else:
              continue
        elif input_t < 0:
            continue
        else:
            continue
  
    return index

In [6]:
etalon1 = Dict.empty(
    key_type=types.unicode_type,
    value_type=types.int_[:,:],)

index = Dict.empty(
    key_type=types.int_,
    value_type=types.int_[:])

[etalon1.update({i:etalon[i]}) for i in etalon.keys()]
test_image = input_images[-4]
index = np.zeros([test_image.shape[1],27])
shape = test_image.shape[1]
t = vers3_dict1(test_image,index,etalon1)

In [31]:
lib_list = [ [[list(etalon.keys())[i],t[j][i],list(etalon.keys())[i]] for i in range(len(t[j])) if (t[j][i] != 0)]  for j in range(len(t)) ] 
for i in range(test_image.shape[1] - 1,-1,-1):
      if i == []:
          continue
      for char_index,column_element in enumerate(lib_list[i]):
        jump = etalon[column_element[0]].shape[1]
        if i - jump < 0:
          continue
        next_char_column = lib_list[i - jump]
        for sosed_index,char_sosed in enumerate(next_char_column):
          if(column_element[0] + char_sosed[0] in bigraph.keys()):
            tupleProb = column_element[1] + char_sosed[1]  + np.log(bigraph[column_element[0] + char_sosed[0]])
            lib_list[i - jump][sosed_index][1] += np.log(-tupleProb)
            lib_list[i - jump][sosed_index][2] += column_element[0]


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]


In [32]:
print(t.shape)
t[-1]


(1333, 27)


array([-407.15774009, -380.22176656, -463.24916213, -252.38104022,
       -375.85444718, -330.72337466, -322.70208579, -267.14177879,
       -337.06098044, -402.15961403, -356.05664937, -474.48364339,
       -478.41018567, -434.44571173, -377.32631593, -346.15385309,
       -359.08448902, -495.26580493, -331.04111642, -580.59989554,
       -349.23153856, -524.91811215, -422.7159307 , -488.4858631 ,
       -376.65501712, -467.30653449, -441.80349652])

In [38]:
maxProb = -1e+16
bestSq =None
for i in lib_list:
  for j in i:
    if j[1] > maxProb:
      maxProb = j[1]
      bestSq = j[2]

In [44]:

print("THE BEST SEQUENCE IN THE WORLD :",bestSq)


THE BEST SEQUENCE IN THE WORLD : ymuwclnryzaevboxtfgkpsdh
